### ex6_spam Spam Classification

We will use SVM to build a spam classifier. We will train a classifier to classify whether a given email x is spam (y=1) or non-spam (y=0). We need to convert each email into a feature vector x. For the purpose of the exercise, we will only use the email body (excluding the email header).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
from scipy.optimize import minimize
from scipy.io import loadmat

### 1: Email Preprocessing

To use an SVM to classify emails into Spam v.s. Non-Spam, you first need to convert each email into a vector of features. In this part, you will implement the preprocessing steps for each email. You should complete the code in processEmail.m to produce a word indices vector for a given email.

1. Lower-casing

2. Stripping HTML

3. Normalizing URLs

4. Normalizing Email address

5. Normalizing Numbers

6. Normalizing dollars

7. Word stemming

8. Removal of non-words

In [2]:
def readFile(filename):
    #READFILE reads a file and returns its entire contents 
    #   file_contents = READFILE(filename) reads a file and returns its entire
    #   contents in file_contents
    #

    # Load File
    try:
        with open(filename, 'r') as openFile:
            file_contents = openFile.read()
    except:
        file_contents = ''
        print('Unable to open {:s}'.format(filename))

    return file_contents


In [9]:
# Extract Features
email=readFile('emailSample1.txt');
print(type(email))
email

<class 'str'>


"> Anyone knows how much it costs to host a web portal ?\n>\nWell, it depends on how many visitors you're expecting.\nThis can be anywhere from less than 10 bucks a month to a couple of $100. \nYou should checkout http://www.rackspace.com/ or perhaps Amazon EC2 \nif youre running something big..\n\nTo unsubscribe yourself from this mailing list, send an email to:\ngroupname-unsubscribe@egroups.com\n\n"

In [13]:
def getVocabList():
    #GETVOCABLIST reads the fixed vocabulary list in vocab.txt and returns a
    #cell array of the words
    #   vocabList = GETVOCABLIST() reads the fixed vocabulary list in vocab.txt 
    #   and returns a cell array of the words in vocabList.


    ## Read the fixed vocabulary list
    with open('vocab.txt', 'r') as vocabFile:

        # Store all dictionary words in dictionary vocabList
        vocabList = {}
        for line in vocabFile.readlines():
            i, word = line.split()
            vocabList[word] = int(i)

    return vocabList


In [15]:
vocabList=getVocabList()

In [29]:
def processEmail(email_contents):
    import re
    from nltk import PorterStemmer
    
#PROCESSEMAIL preprocesses a the body of an email and
#returns a list of word_indices 
#   word_indices = PROCESSEMAIL(email_contents) preprocesses 
#   the body of an email and returns a list of indices of the 
#   words contained in the email. 

# Load Vocabulary
    vocabList = getVocabList();

# Init return value
    word_indices = [];

# ========================== Preprocess Email ===========================

# Find the Headers ( \n\n and remove )
# Uncomment the following lines if you are working with raw emails with the
# full headers

# hdrstart = strfind(email_contents, ([char(10) char(10)]));
# email_contents = email_contents(hdrstart(1):end);

# Lower case
    email_contents = email_contents.lower();

# Strip all HTML
# Looks for any expression that starts with < and ends with > and replace
# and does not have any < or > in the tag it with a space
    email_contents = re.sub('<[^<>]+>', ' ', email_contents);

# Handle Numbers
# Look for one or more characters between 0-9
    email_contents = re.sub('[0-9]+', 'number',email_contents);

# Handle URLS
# Look for strings starting with http:// or https://
    email_contents = re.sub('(http|https)://[^\s]*', 'httpaddr', email_contents);
                           
# Handle Email Addresses
# Look for strings with @ in the middle
    email_contents = re.sub('[^\s]+@[^\s]+', 'emailaddr', email_contents);

# Handle $ sign
    email_contents = re.sub('[$]+', 'dollar', email_contents);

# ========================== Tokenize Email ===========================

# Output the email to screen as well
    print('\n==== Processed Email ====\n\n');

# Process file
    l = 0;
    # Slightly different order from matlab version

    # Split and also get rid of any punctuation
    # regex may need further debugging...
    email_contents = re.split(r'[@$/#.-:&\*\+=\[\]?!(){},\'\'\">_<;%\s\n\r\t]+', email_contents)

    for token in email_contents:

        # Remove any non alphanumeric characters
        token = re.sub('[^a-zA-Z0-9]', '', token)

        # Stem the word 

        token = PorterStemmer().stem(token.strip())

        # Skip the word if it is too short
        if len(token) < 1:
           continue
  

    # Look up the word in the dictionary and add to word_indices if
    # found
    # ====================== YOUR CODE HERE ======================
    # Instructions: Fill in this function to add the index of str to
    #               word_indices if it is in the vocabulary. At this point
    #               of the code, you have a stemmed word from the email in
    #               the variable str. You should look up str in the
    #               vocabulary list (vocabList). If a match exists, you
    #               should add the index of the word to the word_indices
    #               vector. Concretely, if str = 'action', then you should
    #               look up the vocabulary list to find where in vocabList
    #               'action' appears. For example, if vocabList{18} =
    #               'action', then, you should add 18 to the word_indices 
    #               vector (e.g., word_indices = [word_indices ; 18]; ).
    # 
    # Note: vocabList{idx} returns a the word with index idx in the
    #       vocabulary list.
    # 
    # Note: You can use strcmp(str1, str2) to compare two strings (str1 and
    #       str2). It will return 1 only if the two strings are equivalent.
    #
        idx = vocabList[token] if token in vocabList else 0

        # only add entries which are in vocabList
        #   i.e. those with ind ~= 0, 
        #        given that ind is assigned 0 if str is not found in vocabList
        if idx > 0:
            word_indices.append(idx)

        # =============================================================


        # Print to screen, ensuring that the output lines are not too long
        if l + len(token) + 1 > 78:
            print("")
            l = 0
        print('{:s}'.format(token)),
        l = l + len(token) + 1

    # Print footer
    print('\n\n=========================\n')

    return word_indices

Stemmers remove morphological affixes from words, leaving only the word stem.

In [30]:
word_indices  = processEmail(email);

# Print Stats
print('Word Indices: \n');
print( word_indices);


==== Processed Email ====


anyon
know
how
much
it
cost
to
host
a
web
portal
well
it
depend
on
how
mani

visitor
you
re
expect
thi
can
be
anywher
from
less
than
number
buck
a
month

to
a
coupl
of
dollarnumb
you
should
checkout
httpaddr
or
perhap
amazon
ecnumb

if
your
run
someth
big
to
unsubscrib
yourself
from
thi
mail
list
send
an

email
to
emailaddr



Word Indices: 

[86, 916, 794, 1077, 883, 370, 1699, 790, 1822, 1831, 883, 431, 1171, 794, 1002, 1893, 1364, 592, 1676, 238, 162, 89, 688, 945, 1663, 1120, 1062, 1699, 375, 1162, 479, 1893, 1510, 799, 1182, 1237, 810, 1895, 1440, 1547, 181, 1699, 1758, 1896, 688, 1676, 992, 961, 1477, 71, 530, 1699, 531]


### 2. Feature Extraction

We will convert each email into a vector of feature in R^n. You should complete the code in emailFeature.m to produce a feature vector for a given email. 

In [33]:
vocabList = getVocabList();
len(vocabList)

1899

In [55]:
def emailFeatures(word_indices):
#EMAILFEATURES takes in a word_indices vector and produces a feature vector
#from the word indices
#   x = EMAILFEATURES(word_indices) takes in a word_indices vector and 
#   produces a feature vector from the word indices. 

# Total number of words in the dictionary
    n = 1899;

# You need to return the following variables correctly.
    x = np.zeros((n, 1));
    vocabList = getVocabList();
    for i in word_indices:
        if i in vocabList.values():
            x[i,0]=1
    return x

# ====================== YOUR CODE HERE ======================
# Instructions: Fill in this function to return a feature vector for the
#               given email (word_indices). To help make it easier to 
#               process the emails, we have have already pre-processed each
#               email and converted each word in the email into an index in
#               a fixed dictionary (of 1899 words). The variable
#               word_indices contains the list of indices of the words
#               which occur in one email.
# 
#               Concretely, if an email has the text:
#
#                  The quick brown fox jumped over the lazy dog.
#
#               Then, the word_indices vector for this text might look 
#               like:
#               
#                   60  100   33   44   10     53  60  58   5
#
#               where, we have mapped each word onto a number, for example:
#
#                   the   -- 60
#                   quick -- 100
#                   ...
#
#              (note: the above numbers are just an example and are not the
#               actual mappings).
#
#              Your task is take one such word_indices vector and construct
#              a binary feature vector that indicates whether a particular
#              word occurs in the email. That is, x(i) = 1 when word i
#              is present in the email. Concretely, if the word 'the' (say,
#              index 60) appears in the email, then x(60) = 1. The feature
#              vector should look like:
#
#              x = [ 0 0 0 0 1 0 0 0 ... 0 0 0 0 1 ... 0 0 0 1 0 ..];

In [56]:
print('\nExtracting features from sample email (emailSample1.txt)\n');

# Extract Features
features = emailFeatures(word_indices);


Extracting features from sample email (emailSample1.txt)



In [57]:
# Print Stats
print('Length of feature vector: ', len(features));
print('Number of non-zero entries: ', np.sum(features > 0));

Length of feature vector:  1899
Number of non-zero entries:  45


###  3: Train Linear SVM for Spam Classification 
In this section, you will train a linear classifier to determine if an email is Spam or Not-Spam.

In [67]:
# Load the Spam Email dataset
# You will have X, y in your environment

spamTrain=loadmat('spamTrain.mat')
spamTrain.keys()
X=spamTrain['X']
y=spamTrain['y']
print(X.shape)
print(y.shape)

(4000, 1899)
(4000, 1)


In [69]:
from sklearn import svm
import numpy as np

def svmTrain(X, y, C, kernelFunction, tol=1e-3, max_passes=-1, sigma=0.1):
    """Trains an SVM classifier"""

    y = y.flatten() # prevents warning

    # alternative to emulate mapping of 0 -> -1 in svmTrain.m
    #  but results are identical without it
    # also need to cast from unsigned int to regular int
    # otherwise, contour() in visualizeBoundary.py doesn't work as expected
    # y = y.astype("int32")
    # y[y==0] = -1

    if kernelFunction == "gaussian":
        clf = svm.SVC(C = C, kernel="precomputed", tol=tol, max_iter=max_passes, verbose=2)
        return clf.fit(gaussianKernelGramMatrix(X,X, sigma=sigma), y)

    # elif kernelFunction == "linear":
    #     clf = svm.SVC(C = C, kernel="precomputed", tol=tol, max_iter=max_passes, verbose=2)
    #     return clf.fit(np.dot(X,X.T).T, y)

    else: # works with "linear", "rbf"
        clf = svm.SVC(C = C, kernel=kernelFunction, tol=tol, max_iter=max_passes, verbose=2)
        return clf.fit(X, y)

In [71]:
print('\nTraining Linear SVM (Spam Classification)\n')
print('(this may take 1 to 2 minutes) ...\n')

C = 0.1;
model = svmTrain(X, y, C, 'linear');
p_train = model.predict(X);
print('Training Accuracy: ', np.mean(p.flatten() == y.flatten()) * 100);


Training Linear SVM (Spam Classification)

(this may take 1 to 2 minutes) ...

[LibSVM]Training Accuracy:  99.825


In [76]:
spamTest=loadmat('spamTest.mat')
spamTest.keys()
Xtest=spamTest['Xtest']
ytest=spamTest['ytest']

In [85]:
p_test=model.predict(Xtest)
print('Test Accuracy: ', np.mean(p_test.flatten() == ytest.flatten()) * 100);

Test Accuracy:  98.9


### 4: Top Predictors of Spam

Since the model we are training is a linear SVM, we can inspect the weights learned by the model to understand better how it is determining whether an email is spam or not. The following code finds the words with the highest weights in the classifier. Informally, the classifier 'thinks' that these words are the most likely indicators of spam.

In [94]:
w = model.coef_[0]

# from http://stackoverflow.com/a/16486305/583834
# reverse sorting by index
indices = w.argsort()[::-1][:15]
vocabList = sorted(getVocabList().keys())

In [97]:
print('\nTop predictors of spam: \n');
for idx in indices: 
    print( ' {:s} ({:f}) '.format( vocabList[idx], float(w[idx]) ) )


Top predictors of spam: 

 our (0.500614) 
 click (0.465916) 
 remov (0.422869) 
 guarante (0.383622) 
 visit (0.367710) 
 basenumb (0.345064) 
 dollar (0.323632) 
 will (0.269724) 
 price (0.267298) 
 pleas (0.261169) 
 most (0.257298) 
 nbsp (0.253941) 
 lo (0.253467) 
 ga (0.248297) 
 hour (0.246404) 


In [ ]:
# Sort the weights and obtin the vocabulary list
[weight, idx] = sort(model.w, 'descend');
vocabList = getVocabList();

fprintf('\nTop predictors of spam: \n');
for i = 1:15
    fprintf(' %-15s (%f) \n', vocabList{idx(i)}, weight(i));
end

### 5: Try Your Own Emails 

Now that you've trained the spam classifier, you can use it on your own emails! In the starter code, we have included spamSample1.txt,
spamSample2.txt, emailSample1.txt and emailSample2.txt as examples. 
The following code reads in one of these emails and then uses your 
learned SVM classifier to determine whether the email is Spam or 
Not Spam

Set the file to be read in (change this to spamSample2.txt,
emailSample1.txt or emailSample2.txt to see different predictions on
different emails types). Try your own emails as well!

In [114]:
email=readFile('myemail.txt');
word_indices  = processEmail(email);
features = emailFeatures(word_indices);
p1=model.predict(features.T)
print(p1)


==== Processed Email ====


first
releas
sold
out
new
singlefamili
home
releas
in
the
cupertino
union

school
district
resid
number
releas
for
sale
with
quick
close
price
start

from
dollarnumb
number
number
with
the
recent
success
and
sell
out
of
our

first
releas
cupertino
live
work
announc
the
highlyanticip
releas
of
resid

number
cupertino
is
at
the
heart
of
innov
so
it
s
no
coincid
it
s
where
you

ll
find
cupertino
live
work
a
modern
collect
as
uniqu
as
the
address
it
call

home
each
home
ha
been
design
with
it
own
custom
premium
finish
even
more

extraordinari
is
the
bonu
separ
suit
you
will
find
with
each
of
these
home

the
possibl
are
endless
for
thi
uniqu
separ
suit
creativ
work
space
a
bonu

famili
room
entertain
space
librari
think
tank
becaus
thi
is
a
one
of
a
kind

opportun
in
the
highlysought
after
cupertino
union
school
district
don
t
let

someon
els
beat
you
to
the
idea
to
call
thi
home
open
hous
saturdaysunday

number
numbernumb
numberpm
or
call
list
agent
for
privat
